In [1]:
import Pkg; 
cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
pwd()

"/Users/ivanspirandelli/Doktor/Code/MorphoMol/MorphoMolNotebooks"

In [13]:
folder = "../../Data/hpc_out/2_6r7m_oj_0_0_os_0_85/"
folder = "../../Data/collected_simulation_results/2su/6r7m/"

id_index = 1
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, "_")[id_index]) ? parse(Int, split(file, "_")[id_index]) : max
    end
end

min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        @load "$folder$file" input output
        id = parse(Int, split(file, "_")[id_index])
        string = "$(id): $(input["T"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end])"
        evaluation_strings[id] = string
        min_Es[id] = minimum(output["Es"])
    end
end

In [14]:
argmin(min_Es)

10

In [15]:
for elem in evaluation_strings
    println(elem)
end

1: 2.6 | 0.02371 | 8219.57830443252, 8143.38289845842, 8153.410451666697








10: 2.6 | 0.01654 | 8219.578304342, 8139.446858701283, 8160.300327469073


In [10]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(in_out_data.output.Es)
        state = in_out_data.output.states[mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([in_out_data.input.template_radii for _ in 1:in_out_data.input.number_of_molecules]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, in_out_data.input.template_mol), radii, "$(folder)$(id)", in_out_data.input.number_of_molecules, length(in_out_data.input.template_radii))
    catch e
        println(e)
        continue
    end
end

# Print Single Assembly Path

In [38]:
@load "../../Data/collected_simulation_results/2su/6r7m/10_6r7m.jld2" input output


2-element Vector{Symbol}:
 :input
 :output

In [39]:
input["template_centers"]

3×1206 Matrix{Float64}:
 -27.8796   -26.6674   -26.5597   …  -31.3903   -31.5191   -32.6934
   9.063      9.54921    9.03875       8.50467    8.25724    9.091
  -4.79258   -4.1493    -2.71995      -1.62113   -0.2168    -2.14403

In [41]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end


# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
